In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import librosa
import numpy as np 
import IPython.display as ipd
import librosa.display
from matplotlib import *
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import glob
from sklearn.pipeline import make_pipeline
from tensorflow.keras.utils import to_categorical

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score , classification_report , confusion_matrix  ,roc_curve , roc_auc_score , accuracy_score
from sklearn.metrics import precision_recall_curve , auc ,f1_score , plot_confusion_matrix , precision_score , recall_score
from sklearn.datasets import load_digits

In [68]:
file_name = []
labels = []
for i in glob.glob('/kaggle/input/cremad/'+'*/*.wav'):
    labels.append(i.split('_')[-2])
    file_name.append(i)

In [69]:
df = pd.DataFrame({'FileName':file_name,'Label':labels})
df
# df = df.sample(100)
# df.reset_index(drop=True,inplace=True)

,FileName,Label
0,/kaggle/input/cremad/AudioWAV/1028_TSI_DIS_XX.wav,DIS
1,/kaggle/input/cremad/AudioWAV/1075_IEO_HAP_LO.wav,HAP
2,/kaggle/input/cremad/AudioWAV/1084_ITS_HAP_XX.wav,HAP
3,/kaggle/input/cremad/AudioWAV/1067_IWW_DIS_XX.wav,DIS
4,/kaggle/input/cremad/AudioWAV/1066_TIE_DIS_XX.wav,DIS
...,...,...
7437,/kaggle/input/cremad/AudioWAV/1060_IEO_ANG_MD.wav,ANG
7438,/kaggle/input/cremad/AudioWAV/1088_IWL_ANG_XX.wav,ANG
7439,/kaggle/input/cremad/AudioWAV/1050_IOM_ANG_XX.wav,ANG
7440,/kaggle/input/cremad/AudioWAV/1044_IWL_SAD_XX.wav,SAD


In [70]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features


In [71]:
for index_num,row in tqdm(df.iterrows()):
    print(row[0])
    break

0it [00:00, ?it/s]

/kaggle/input/cremad/AudioWAV/1028_TSI_DIS_XX.wav


In [27]:
extracted_features=[]
for index_num,row in tqdm(df.iterrows()):
    file_name = row[0]
#   file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    final_class_labels=row[1]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

7442it [07:32, 16.43it/s]


In [28]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-435.88214, 139.41362, 2.123767, 79.77228, -1...",DIS
1,"[-433.89142, 143.1926, 9.697038, 50.856354, -1...",HAP
2,"[-421.8806, 124.63293, 6.897354, 53.84116, -2....",HAP
3,"[-379.7409, 131.38957, 25.26957, 46.059532, -5...",DIS
4,"[-398.7406, 146.85852, 1.4794314, 50.891872, -...",DIS


In [34]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())
### Label Encoding -> Label Encoder
from tensorflow.keras.utils import to_categorical
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))
### Train Test Split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [73]:
X_train.shape, y_train.shape, x_test.shape, y_test.shape

((5953, 40), (5953, 6), (1489, 40), (1489, 6))

In [36]:
#Creating Different Models

In [37]:
model_list = []
accuracy_list = []
recall_list = []
precision_list = []
f1_score_list= [] 
cross_val_score_list = []

In [53]:
def Model_features(X_train , y_train , X_test , y_test , y_pred , classifier  , model_name):
#     fig ,ax = plt.subplots(figsize = (7,6))
    accuracy = round(accuracy_score(y_test , y_pred) , 3)
    precision  = round(precision_score(y_test, y_pred, average="micro")  ,3)
    recall = round(recall_score(y_test , y_pred, average="micro") ,3)
    f1_s = round(f1_score(y_test , y_pred,average="micro") , 3)
    c_val_score = round(cross_val_score(classifier , X , y , cv=10 , scoring='accuracy').mean()  , 3)
#     accuracy , precision , recall , f1_s ,c_val_score= round(accuracy_score(y_test , y_pred) , 3) , round(precision_score(y_test, y_pred, average="micro")  ,3), round(recall_score(y_test , y_pred) ,3), round(f1_score(y_test , y_pred) , 3) , round(cross_val_score(classifier , X , y , cv=10 , scoring='accuracy').mean()  , 3)
    
    print(f'Accuracy Score is :{accuracy}')
    print(f'Precision Score is :{precision}')
    print(f'Recall Score is :{recall}')
    print(f'f1  Score is :{f1_s}')
    print(f'cross val score is :{c_val_score}')
    model_list.append(model_name)
    accuracy_list.append(accuracy)
    recall_list.append(recall)
    precision_list.append(precision)
    f1_score_list.append(f1_s)
    cross_val_score_list.append(c_val_score)
    
#     print(f'f1  Score is :{round(specificity_score(y_test , y_pred) , 3)}')
    print(metrics.classification_report(y_test, y_pred))

In [54]:
# Define a function that plots the feature weights for a classifier.
def feature_weights(X_df, classifier, classifier_name):
    weights = round(pd.Series(classifier.coef_[0], index=X_df.columns.values).sort_values(ascending=False) ,2 )
    
    top_weights_selected = weights[:5]
    plt.figure(figsize=(7,6))
    plt.tick_params(labelsize=10)#plt.xlabel(fontsize=10)
    plt.title(f'{classifier_name} - Top 5 Features')
    ax = top_weights_selected.plot(kind="bar")
    ax.bar_label(ax.containers[0])
    
    return print("")

In [55]:
# Random forest Model

In [57]:
model_rf=RandomForestClassifier()
model_rf.fit(X_train,y_train)
y_pred = model_rf.predict(x_test)
model_rf.score(x_test , y_test) 

Model_features(x_train , y_train , x_test , y_test  , y_pred , model_rf , "Random Forest Classifier") 

Accuracy Score is :0.131
Precision Score is :0.786
Recall Score is :0.131
f1  Score is :0.225
cross val score is :0.126
              precision    recall  f1-score   support

           0       0.81      0.44      0.57       270
           1       0.00      0.00      0.00       260
           2       0.00      0.00      0.00       246
           3       1.00      0.00      0.01       243
           4       0.60      0.02      0.03       197
           5       0.76      0.26      0.39       273

   micro avg       0.79      0.13      0.22      1489
   macro avg       0.53      0.12      0.17      1489
weighted avg       0.53      0.13      0.18      1489
 samples avg       0.13      0.13      0.13      1489



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [2]:
# svm = SVC(kernel='rbf', probability=True) 
# svm.fit(x_train_new,y_train_new)

# # Make predictions (classes and probabilities) with the trained model on the test set.
# y_pred = svm.predict(x_test)
# svm.score(x_test , y_test)  

# Model_features(x_train , y_train , x_test , y_test  , y_pred , svm , "Support Vector Machine") 

In [77]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_pred = dtc.predict(x_test)
dtc.score(x_test , y_test) 

Model_features(x_train , y_train , x_test , y_test  , y_pred , dtc , "Decision Tree Classifier") 

Accuracy Score is :0.336
Precision Score is :0.336
Recall Score is :0.336
f1  Score is :0.336
cross val score is :0.313
              precision    recall  f1-score   support

           0       0.55      0.54      0.54       270
           1       0.24      0.23      0.24       260
           2       0.24      0.24      0.24       246
           3       0.27      0.26      0.27       243
           4       0.23      0.29      0.26       197
           5       0.47      0.42      0.44       273

   micro avg       0.34      0.34      0.34      1489
   macro avg       0.33      0.33      0.33      1489
weighted avg       0.34      0.34      0.34      1489
 samples avg       0.34      0.34      0.34      1489



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Naive Bayes model

In [3]:
# nbm = GaussianNB()
# nbm.fit(X_train, y_train)
# y_pred = nbm.predict(X_test)
# nbm.score(x_test , y_test)
# Model_features(x_train , y_train , x_test , y_test  , y_pred , dtc , "Naive Bayes") 

In [4]:
# model_lr = LogisticRegression(random_state = 0)
# model_lr.fit(X_train, y_train)
# y_pred = model_lr(x_test)
# model_lr.score(x_test , y_test)
# Model_features(x_train , y_train , x_test , y_test  , y_pred , dtc , "LogisticRegression") 


In [5]:
# model_ada = AdaBoostClassifier()
# model_ada.fit(X_train, y_train)
# y_pred = model_ada.predict(x_test)
# model_ada.score(x_test , y_test) 

# Model_features(x_train , y_train , x_test , y_test  , y_pred , dtc , "AdaBoost Classifier") 

In [6]:
# model_gbc = GradientBoostingClassifier().fit(X_train, y_train)
# y_pred = model_gbc.predict(x_test)
# model_gbc.score(x_test , y_test)

Model Performance

In [ ]:
dict = {'Model':model_list, 'Accuracy':accuracy_list , 'Precision':precision_list , 'f1_score':f1_score_list , 'Recall':recall_list ,'Cross_Val_Score':cross_val_score_list}
model_df = pd.DataFrame(dict).sort_values(ascending = False , by = 'Accuracy')
model_df 

In [ ]:
# import time

# starting_time = time.time()

# svclassifier.fit(X_train, y_train)
# print("Trained model in %s ms " % str(time.time() - starting_time))